meta-llama/Llama-3.2-1B-Instruct

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
# Set device to CPU since we don't have a GPU
device = torch.device("cpu")

# Load tokenizer and model (using LLaMA 7B as an example)
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model using accelerate for CPU optimization
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", offload_folder="offload").to(device)

model.safetensors:  34%|###4      | 849M/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
# Example: List of customer questions (can be extended or read from a file)
customer_questions = [
    "What are the amenities available at the hotel?",
    "Does the hotel offer free Wi-Fi?",
    "Are pets allowed at the hotel?",
    "What is the check-in and check-out time?",
    "Is breakfast included in the room rate?",
    "Are there any family rooms available?",
    "Is there a swimming pool at the hotel?",
    "Does the hotel provide airport shuttle services?",
    "Are there any local attractions nearby?",
    "What are the payment options at the hotel?",
]


In [5]:
# Function to prepare a prompt
def prepare_prompt(questions):
    prompt = ("You are analyzing customer questions about a hotel property. "
              "Based on these questions, what recommendations can be made to change the  hotel property details page to address customer concerns and enhance user experience\n\n")
    prompt += "\n".join(f"- {q}" for q in questions)
    return prompt



In [6]:

# Ensure the tokenizer has a pad token
if tokenizer.pad_token is None:
    # Option 1: Use eos_token as pad_token if the tokenizer doesn't have a pad token
    tokenizer.pad_token = tokenizer.eos_token

    # Option 2: Alternatively, you can add a new [PAD] token
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def generate_suggestions(prompt):
    # Tokenize the input prompt with padding and return the attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Ensure the `pad_token_id` is set
    pad_token_id = tokenizer.pad_token_id
    
    # Generate the response using the model
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],  # Provide the attention mask
            max_length=800,  # Increase max length for longer suggestions
            temperature=0.7,
            pad_token_id=pad_token_id,  # Use the correct pad token ID
            num_return_sequences=1
        )
    
    # Decode the model output into human-readable text
    suggestions = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return suggestions




In [7]:
small_batch = customer_questions[:5]  # Testing with a small batch
prompt = prepare_prompt(small_batch)
print("Prompt:\n", prompt)  # See the input prompt

# Generate content improvement suggestions
suggestions = generate_suggestions(prompt)
print("Content Improvement Suggestions:\n", suggestions)


Prompt:
 You are analyzing customer questions about a hotel property. Based on these questions, what recommendations can be made to change the  hotel property details page to address customer concerns and enhance user experience

- What are the amenities available at the hotel?
- Does the hotel offer free Wi-Fi?
- Are pets allowed at the hotel?
- What is the check-in and check-out time?
- Is breakfast included in the room rate?


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Content Improvement Suggestions:
 You are analyzing customer questions about a hotel property. Based on these questions, what recommendations can be made to change the  hotel property details page to address customer concerns and enhance user experience

- What are the amenities available at the hotel?
- Does the hotel offer free Wi-Fi?
- Are pets allowed at the hotel?
- What is the check-in and check-out time?
- Is breakfast included in the room rate? 
- Is the hotel located near a public transportation hub?
- Is the hotel's proximity to a major airport easily accessible?
- What are the hotel's room rates and packages?
- Are there any special promotions or discounts available?

Based on the customer questions, here are some potential recommendations to change the hotel property details page:

**Amenities and Services**

* Add a section highlighting the hotel's amenities, such as the pool, fitness center, and spa.
* Include information about any additional services, like concierge or b